# PyPSA Components Basics

This tutorial covers the basic components available in PyPSA and how to configure them. We'll explore generators, loads, lines, and other essential components.

## Basic Components Overview

PyPSA represents power and energy systems using the following components:

1. **Network**: The main container for all components
2. **Bus**: A node in the network where components attach
3. **Carrier**: Energy carrier or technology (e.g., electricity, hydrogen, gas, coal, oil, biomass, on-/offshore wind, solar)
4. **Load**: Energy consumer (e.g., electricity demand)
5. **Generator**: Power plant or generation unit (e.g., power plant, wind turbine, PV panel)
6. **Line**: Power distribution and transmission lines (overhead and cables)
7. **Link**: DC transmission lines or other connections
8. **Store**: Storage units that can charge and discharge
9. **Transformer**: Components that transform between different voltage levels

Let's create a network and explore each component type:

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional

# Create network with time series
network = pypsa.Network()
snapshots = pd.date_range("2024-01-01", periods=24, freq="h")
network.set_snapshots(snapshots)

# Add buses with nominal voltage of 20 kV
# Note: All voltages in PyPSA are in kV
for i in range(3):
    network.add("Bus", f"bus {i}", v_nom=20.0)

# Add carriers with CO2 emissions and nice names for plotting
network.add(
    "Carrier",
    ["gas", "solar", "wind", "coal"],
    co2_emissions=[0.2, 0.0, 0.0, 0.3],  # tCO2/MWh
    nice_name=["Natural Gas", "Solar PV", "Wind", "Coal"],
    color=["indianred", "yellow", "dodgerblue", "grey"]
)

print("Buses in the network:")
print(network.buses)

print("\nCarriers in the network:")
print(network.carriers)

## Generators

Generators represent power plants or other generation units. They can be configured with various parameters:

- `p_nom`: Nominal power capacity (MW)
- `p_nom_extendable`: Whether capacity can be expanded
- `marginal_cost`: Operating cost per MWh
- `capital_cost`: Investment cost per MW
- `efficiency`: Conversion efficiency
- `p_max_pu`: Maximum power output as fraction of p_nom
- `p_min_pu`: Minimum power output as fraction of p_nom
- `ramp_limit_up`: Maximum ramp up rate
- `ramp_limit_down`: Maximum ramp down rate

In [ ]:
# Add a gas generator with unit commitment
network.add(
    "Generator",
    "gas_plant",
    bus="bus 0",
    p_nom=100,  # Nominal power capacity in MW
    p_nom_extendable=True,  # Allow capacity expansion
    marginal_cost=50,  # Operating cost per MWh
    capital_cost=1000,  # Investment cost per MW
    carrier="gas",
    efficiency=0.4,  # Conversion efficiency
    p_min_pu=0.3,  # Minimum power output as fraction of p_nom
    p_max_pu=1.0,  # Maximum power output as fraction of p_nom
    ramp_limit_up=0.2,  # Maximum ramp up rate
    ramp_limit_down=0.2  # Maximum ramp down rate
)

# Add a solar generator with variable output
solar_profile = pd.Series(
    [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
     0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    index=network.snapshots
)

network.add(
    "Generator",
    "solar",
    bus="bus 1",
    p_nom=50,
    p_nom_extendable=True,
    p_max_pu=solar_profile,
    marginal_cost=0,
    capital_cost=800,
    carrier="solar"
)

print("\nGenerators in the network:")
print(network.generators)

## Loads

Loads represent electricity demand. They can have fixed or time-varying demand profiles:

- `p_set`: Power demand (fixed or time series)
- `q_set`: Reactive power demand (optional)
- `e_annual`: Annual energy demand (MWh)

In [ ]:
# Create a load profile
load_profile = pd.Series(
    [0.6, 0.5, 0.4, 0.4, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.7,
     0.7, 0.7, 0.7, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.6, 0.5, 0.5],
    index=network.snapshots
)

# Add loads with different profiles
network.add(
    "Load",
    "load_1",
    bus="bus 1",
    p_set=50 * load_profile,
    e_annual=50 * load_profile.sum()
)

network.add(
    "Load",
    "load_2",
    bus="bus 2",
    p_set=30,
    e_annual=30 * 24 * 365
)

print("\nLoads in the network:")
print(network.loads)

## Lines and Links

Lines represent AC transmission connections, while Links can represent DC lines or other connections:

### Line Parameters:
- `s_nom`: Nominal apparent power (MVA)
- `x`: Reactance (p.u.)
- `r`: Resistance (p.u.)
- `g`: Conductance (p.u.)
- `b`: Susceptance (p.u.)
- `capital_cost`: Investment cost per MVA
- `length`: Line length (km)
- `type`: Line type (e.g., "AC", "DC")

### Link Parameters:
- `p_nom`: Nominal power capacity (MW)
- `efficiency`: Conversion efficiency
- `capital_cost`: Investment cost per MW
- `marginal_cost`: Operating cost per MWh

In [ ]:
# Add AC transmission lines
for i in range(3):
    network.add(
        "Line",
        f"line_{i}",
        bus0=f"bus {i}",
        bus1=f"bus {(i + 1) % 3}",
        x=0.1,
        r=0.01,
        g=0.0,
        b=0.0,
        s_nom=100,
        s_nom_extendable=True,
        capital_cost=1000,
        length=100,
        type="AC"
    )

# Add a DC link
network.add(
    "Link",
    "dc_link",
    bus0="bus 0",
    bus1="bus 2",
    p_nom=50,
    p_nom_extendable=True,
    capital_cost=2000,
    efficiency=0.95,
    marginal_cost=0.1
)

print("\nLines in the network:")
print(network.lines)

print("\nLinks in the network:")
print(network.links)

## Stores

Stores represent storage units that can charge and discharge:

- `e_nom`: Nominal energy capacity (MWh)
- `e_nom_extendable`: Whether energy capacity can be expanded
- `e_cyclic`: Whether storage can be cycled
- `e_min_pu`: Minimum energy level as fraction of e_nom
- `p_nom`: Nominal power capacity (MW)
- `p_nom_extendable`: Whether power capacity can be expanded
- `capital_cost_e`: Investment cost per MWh
- `capital_cost_p`: Investment cost per MW
- `marginal_cost`: Operating cost per MWh
- `standing_loss`: Self-discharge rate per hour
- `efficiency_dispatch`: Efficiency when discharging
- `efficiency_store`: Efficiency when charging

In [ ]:
# Add a storage unit
network.add(
    "Store",
    "battery",
    bus="bus 1",
    e_nom=100,
    e_nom_extendable=True,
    e_cyclic=True,
    e_min_pu=0.1,
    p_nom=20,
    p_nom_extendable=True,
    capital_cost_e=100,
    capital_cost_p=1000,
    marginal_cost=0.1,
    standing_loss=0.01,
    efficiency_dispatch=0.95,
    efficiency_store=0.95
)

print("\nStores in the network:")
print(network.stores)

## Transformers

Transformers are used to connect buses with different voltage levels:

- `s_nom`: Nominal apparent power (MVA)
- `x`: Reactance (p.u.)
- `r`: Resistance (p.u.)
- `tap_ratio`: Transformer tap ratio
- `phase_shift`: Phase shift angle (degrees)
- `type`: Transformer type

In [ ]:
# Add a transformer
network.add(
    "Transformer",
    "transformer_1",
    bus0="bus 0",
    bus1="bus 1",
    s_nom=50,
    x=0.1,
    r=0.01,
    tap_ratio=1.0,
    phase_shift=0,
    type="AC"
)

print("\nTransformers in the network:")
print(network.transformers)

## Component Groups

PyPSA allows grouping components for easier management and analysis:

- `group`: Group name for the component
- `carrier`: Energy carrier type
- `type`: Component type
- `sub_network`: Sub-network identifier

In [ ]:
# Add components with groups
network.add(
    "Generator",
    "wind_1",
    bus="bus 2",
    p_nom=30,
    carrier="wind",
    group="renewables",
    type="onshore"
)

network.add(
    "Generator",
    "solar_2",
    bus="bus 2",
    p_nom=20,
    carrier="solar",
    group="renewables",
    type="utility"
)

print("\nGenerators grouped by carrier:")
print(network.generators.groupby("carrier").size())

## Next Steps

In the next tutorial, we'll learn how to:
- Visualize and analyze these components in the network
- Perform power flow analysis
- Optimize the network operation
- Analyze network constraints and congestion
- Implement contingency analysis